## Google authentication

In [97]:
import os
import tensorflow as tf
import json
from google.colab import auth
auth.authenticate_user()

use_tpu = 'COLAB_TPU_ADDR' in os.environ
if use_tpu:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)
  
  with tf.Session(TPU_ADDRESS) as session:
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.
else:
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

TPU address is grpc://10.5.7.250:8470


## Download *improv* project

In [98]:
import sys

!test -d improv || git clone https://github.com/rikhuijzer/improv improv
if not 'improv' in sys.path:
  sys.path += ['improv']
%cd improv
!git log -n 1 --format='commit %H'
%cd ..

/content/improv
commit 154a61b5128e2d90fded8000eca3c7c630390bc6
/content


In [99]:
!test -d nlu_datasets || git clone https://github.com/rikhuijzer/nlu_datasets nlu_datasets
if not 'nlu_datasets' in sys.path:
  sys.path += ['nlu_datasets']
%cd nlu_datasets
# !git reset --hard 12e6fe28bf4ad4ba9ebc17f6a1c053ff9de62b5e
!git log -n 1 --format='commit %H'
%cd ..

/content/nlu_datasets
commit eb4d22e887452085e0c69f742edcbdc0c2e6e7cd
/content


In [100]:
!pip install rasa_nlu==0.13.8

## BERT pre-trained 


In [0]:
bucket = 'benchmark-tpu-bucket' #@param {type:"string"}
assert bucket, 'Must specify an existing GCS bucket name'
bucket_models_dir = 'gs://{}/bert/models'.format(bucket)
bert_model = 'BERT-Large, Uncased' #@param ['BERT-Base, Uncased', 'BERT-Large, Uncased', 'BERT-Large, Cased', 'BERT-Large, Multilingual Cased']

# lowest two are unavailable in cloud-tpu-checkpoints
# need to rewrite to pull them from google storage apis
bert_model_map = {
    'BERT-Base, Uncased': 'uncased_L-12_H-768_A-12',
    'BERT-Large, Uncased': 'uncased_L-24_H-1024_A-16',
    'BERT-Large, Cased': 'cased_L-24_H-1024_A-16',
    'BERT-Large, Multilingual Cased': 'multi_cased_L-12_H-768_A-12'
}
bert_model_mapped = bert_model_map[bert_model]

bert_pretrained_dir = 'gs://cloud-tpu-checkpoints/bert/' + bert_model_mapped

In [0]:
from improv.config import HParams
from improv.utils import get_project_root
from pathlib import Path
from datetime import datetime, timedelta

corpus = 'webapplications'  #@param ['chatbot', 'askubuntu', 'webapplications', 'snips2017']
task = 'ner' #@param ['ner', 'intent', 'ner_intent']
num_train_steps = 600  #@param {type:"integer"}
train_batch_size = 32  #@param {type:"integer"}
iterations_per_loop = 200  #@param {type:"integer"}
new_dir = True  #@param {type:"boolean"}
if new_dir:
  output_dir_name = str(datetime.now() + timedelta(hours=1))[:-7]
tpu_name = 'grpc://' + os.environ['COLAB_TPU_ADDR'] if use_tpu else ''
do_train = True  #@param {type:"boolean"}
do_eval = True  #@param {type:"boolean"}
do_predict = True  #@param {type:"boolean"}

h_params = HParams(
  data_dir=Path('nlu_datasets') / 'generated' / corpus / task,
  bert_config_file=os.path.join(bert_pretrained_dir, 'bert_config.json'),
  task=task,
  task_name=corpus,
  vocab_file=os.path.join(bert_pretrained_dir, 'vocab.txt'),
  output_dir=bucket_models_dir + '/{}/{}'.format(corpus, output_dir_name),
  local_dir=str(get_project_root() / 'tmp' / corpus / output_dir_name),
  init_checkpoint=os.path.join(bert_pretrained_dir, 'bert_model.ckpt'),
  do_lower_case=bert_model.startswith('uncased'),
  max_seq_length=128,
  do_train_eval=False,
  do_train=do_train,
  do_eval=do_eval,
  do_predict=do_predict,
  train_batch_size=train_batch_size,
  eval_batch_size=8,
  predict_batch_size=8,
  learning_rate=5e-5,
  num_train_steps=num_train_steps,
  warmup_proportion=0.1,
  save_checkpoints_steps=50,  # ignored by TPUEstimator
  save_summary_steps=50,  # ignored by TPUEstimator
  iterations_per_loop=iterations_per_loop,  # ignored by TPUEstimator, or is it?
  use_tpu=use_tpu,
  tpu_name=tpu_name,
  tpu_zone=None,
  gcp_project=None,
  master=None,
  num_tpu_cores=8
)

tf.gfile.MakeDirs(h_params.output_dir)
tf.gfile.MakeDirs(h_params.local_dir)

## Define params

## Main


In [95]:
from improv.kyzhouhzau_ner import main, run

result = run(h_params)

INFO:tensorflow:loss = 0.00043892654, step = 400 (103.264 sec)
INFO:tensorflow:global_step/sec: 1.93684
INFO:tensorflow:examples/sec: 61.9788
INFO:tensorflow:Enqueue next (200) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (200) batch(es) of data from outfeed.
INFO:tensorflow:Saving checkpoints for 600 into gs://benchmark-tpu-bucket/bert/models/webapplications/2019-01-26 20:21:42/model.ckpt.
INFO:tensorflow:loss = 0.00031148695, step = 600 (99.346 sec)
INFO:tensorflow:global_step/sec: 2.01314
INFO:tensorflow:examples/sec: 64.4204
INFO:tensorflow:Stop infeed thread controller
INFO:tensorflow:Shutting down InfeedController thread.
INFO:tensorflow:InfeedController received shutdown signal, stopping.
INFO:tensorflow:Infeed thread finished, shutting down.
INFO:tensorflow:infeed marked as finished
INFO:tensorflow:Stop output thread controller
INFO:tensorflow:Shutting down OutfeedController thread.
INFO:tensorflow:OutfeedController received shutdown signal, stopping.
INFO:tensorfl

In [96]:
# in need of refactoring into improv
if h_params.task == 'intent':
  from improv.utils import convert_result_pred
  from sklearn.metrics import f1_score
  from improv.my_classifier import get_unique_intents, get_data_filename, get_intents

  def print_intent_f1(h_params: HParams, result):
    intents = get_unique_intents(get_data_filename(h_params))

    y_pred = []
    for pred in result:
      probs = pred['probabilities']
      id = [i for i, j in enumerate(probs) if j == max(probs)][0]
      y_pred.append(intents[id])

    y_true = get_intents(get_data_filename(h_params), training=False)
    score = f1_score(y_true, y_pred, average='weighted')
    score = round(score, 3)
    print('intents weighted f1: {}'.format(score))

  print_intent_f1(h_params, result)
else:
  from improv.kyzhouhzau_ner import evaluate_ner_pred_result

  evaluate_ner_pred_result(h_params, result)
  
  from pathlib import Path
  from improv.evaluate import print_scores

  print_scores(Path(h_params.local_dir) / 'results.txt')

text: ['alternative', 'to', 'facebook']
true: ['O', 'O', 'B-WebService']
pred: ['O', 'O', 'B-WebService']


text: ['how', 'do', 'i', 'delete', 'my', 'facebook', 'account', '?']
true: ['O', 'O', 'O', 'O', 'O', 'B-WebService', 'O', 'O']
pred: ['O', 'O', 'O', 'O', 'X', 'O', 'B-WebService', 'O']


text: ['are', 'there', 'any', 'good', 'pandora', 'alternatives', 'with', 'general', 'availability', 'outside', 'the', 'us', '?']
true: ['O', 'O', 'O', 'O', 'B-WebService', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred: ['O', 'O', 'O', 'O', 'B-WebService', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


text: ['is', 'it', 'possible', 'to', 'export', 'my', 'data', 'from', 'trello', 'to', 'back', 'it', 'up', '?']
true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-WebService', 'O', 'O', 'O', 'O', 'O']
pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O']


text: ['is', 'there', 'an', 'online', 'alternative', 'to', 'igoogle']
true: ['O', 'O', 'O', 'O', 'O', 'O', 'B-WebService']
pred: ['

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
